# Mergear shps para crear las capas de CartoDB

CartoDB permite el uso de sólo 4 capas para sus usuarios básicos, de manera que se mergearán los shps creando 4 capas principales: 
    1. Divisiones políticas de la CABA y sus indicadores (radios, fracciones, barrios y comunas)
    2. Estaciones de subte, metrobús y ferrocarril.
    3. Líneas de subte, metrobús, ferrocarril y colectivos.
    4. Buffers de los points y sus indicadores.

In [7]:
import os
from scripts.path_finders import get_indicators_shp_path, get_indicators_dir, get_indicators_path
from scripts.path_finders import get_transport_shp_path
from scripts.cartodb_layers import merge_shapefiles
import shapefile
from scripts.create_indicators import get_or_create_indicators_df
from scripts import recalculate_indicators

## 1. Divisiones políticas de la CABA y sus indicadores (radios, fracciones, barrios y comunas)

In [8]:
paths_1 = {
    "RADIO": get_indicators_shp_path("radios_censo_2010"),
    "BARRIO": get_indicators_shp_path("barrios_censo_2010"),
    "DPTO": get_indicators_shp_path("comunas_censo_2010"),
    "FRAC": get_indicators_shp_path("fracciones_censo_2010")
}
divs_path = os.path.join(get_indicators_dir(), "divisiones", "divisiones")

In [9]:
merge_shapefiles(paths_1, divs_path)

Merging FRAC            fracciones_censo_2010
Merging DPTO            comunas_censo_2010
Merging RADIO           radios_censo_2010
Merging BARRIO          barrios_censo_2010


Recalculamos todos los indicadores por barrio.

In [3]:
barrios_shp_dir = "shp/divisiones/barrios_censo_2010"
recalculate_indicators.recalculate_indicators(barrios_shp_dir, "RADIO", skip=["14_10_2"])

Chequeamos que los indicadores calculados por barrio sean consistentes con el total de la CABA.

In [4]:
shp_indic_barrios = "indicadores/barrios_censo_2010/barrios_censo_2010"
shp_indic_comunas = "indicadores/comunas_censo_2010/comunas_censo_2010"
shp_indic_fracciones = "indicadores/fracciones_censo_2010/fracciones_censo_2010"
shp_indic_radios = "indicadores/radios_censo_2010/radios_censo_2010"

In [5]:
def total_caba_population(shp_indicators):
    sf = shapefile.Reader(shp_indicators)
    pop_index = [field[0] for field in sf.fields[1:]].index("hab")
    return sum((record[pop_index] for record in sf.iterRecords() if record[pop_index] is not None))

In [6]:
print total_caba_population(shp_indic_barrios)
print total_caba_population(shp_indic_comunas)
print total_caba_population(shp_indic_fracciones)
print total_caba_population(shp_indic_radios)

print "Margen de error", total_caba_population(shp_indic_barrios) / total_caba_population(shp_indic_comunas) - 1

2889007.41538
0.0
2890151.0
2890151.0
Margen de error

ZeroDivisionError: float division by zero

## 2. Estaciones de subte, metrobús y ferrocarril.

In [114]:
paths_2 = {
    "est_sub_acc": get_transport_shp_path("estaciones-accesibles"),
    "est_sub_prem": get_transport_shp_path("estaciones-premetro"),
    "est_ffcc": get_transport_shp_path("estaciones-ferrocarril"),
    "est_subte": get_transport_shp_path("subte-estaciones"),
    "est_metrobus": get_transport_shp_path("metrobus-estaciones")
}
est_path = os.path.join(get_indicators_dir(), "estaciones", "estaciones")

In [115]:
merge_shapefiles(paths_2, est_path)

Merging est_metrobus    estaciones_de_metrobus
Merging est_ffcc        estaciones_de_ferrocarril
Merging est_sub_acc     accesibilidad_en_estaciones_de_subte
Merging est_sub_prem    estaciones_de_premetro
Merging est_subte       estaciones_de_subte


## 3. Líneas de subte, metrobús, ferrocarril y colectivos.

In [116]:
paths_3 = {
    "lin_colectivos": get_transport_shp_path("recorridos_de_colectivos"),
    "lin_sub_prem": get_transport_shp_path("recorrido-premetro"),
    "lin_metrobus": get_transport_shp_path("metrobus-recorrido"),
    "lin_ffcc": get_transport_shp_path("linea-ferrocarril"),
    "lin_subte": get_transport_shp_path("subte-lineas")
}
lines_path = os.path.join(get_indicators_dir(), "lineas", "lineas")

In [117]:
merge_shapefiles(paths_3, lines_path)

Merging lin_ffcc        linea ferrocarril_1991
Merging lin_metrobus    recorrido_de_metrobus
Merging lin_subte       red_de_subte
Merging lin_sub_prem    red_de_premetro
Merging lin_colectivos  recorridos_de_colectivos


## 4. Buffers de las estaciones y sus indicadores.

In [118]:
replacements = {
    "accesibilidad_en_estaciones_de_subte": "est_sub_acc",
    "estaciones_de_ferrocarril": "est_ffcc",
    "estaciones_de_metrobus": "est_metrobus",
    "estaciones_de_premetro": "est_sub_prem",
    "estaciones_de_subte": "est_subte"
}
buffers_dir = os.path.join(get_indicators_dir(), "buffers")
buffers_est_path = os.path.join(get_indicators_dir(), "buffers_estaciones", "buffers_estaciones")

In [119]:
merge_shapefiles(buffers_dir, buffers_est_path, replacements=replacements)

Merging est_sub_prem-buffer2000 estaciones_de_premetro-buffer2000
Merging est_ffcc-buffer2000 estaciones_de_ferrocarril-buffer2000
Merging est_metrobus-buffer500 estaciones_de_metrobus-buffer500
Merging est_ffcc-buffer1000 estaciones_de_ferrocarril-buffer1000
Merging est_sub_prem-buffer1000 estaciones_de_premetro-buffer1000
Merging est_metrobus-buffer2000 estaciones_de_metrobus-buffer2000
Merging est_subte-buffer1500 estaciones_de_subte-buffer1500
Merging est_sub_prem-buffer1500 estaciones_de_premetro-buffer1500
Merging est_sub_acc-buffer1500 accesibilidad_en_estaciones_de_subte-buffer1500
Merging est_subte-buffer2000 estaciones_de_subte-buffer2000
Merging est_sub_acc-buffer500 accesibilidad_en_estaciones_de_subte-buffer500
Merging est_subte-buffer500 estaciones_de_subte-buffer500
Merging est_sub_prem-buffer500 estaciones_de_premetro-buffer500
Merging est_sub_acc-buffer1000 accesibilidad_en_estaciones_de_subte-buffer1000
Merging est_sub_acc-buffer2000 accesibilidad_en_estaciones_de_sub